## FIRMS API configuration

In [18]:
API_KEY = "b3b38f287a8ba9db57fac29012ed1e9c"
FIRMS_URL = f"https://firms.modaps.eosdis.nasa.gov/api/country/csv/{API_KEY}/VIIRS_NOAA21_NRT/USA/2"

In [13]:
import pandas as pd
from ipyleaflet import Map,GeoData
from apscheduler.schedulers.background import BackgroundScheduler
from IPython.display import display, clear_output
import geopandas
# Create a map centered around the USA
map = Map(center=(37.0902, -95.7129), zoom=4)

## Function to fetch and process data into a DataFrame

In [14]:
def fetch_and_process_fire_data():
    df=pd.read_csv(FIRMS_URL)
    df_subset= df[['latitude','longitude','bright_ti4','acq_date','acq_time','confidence','frp']]
    filtered_df=df_subset[df_subset['confidence']!='l']
    geodf=geopandas.GeoDataFrame(
    filtered_df, geometry=geopandas.points_from_xy(filtered_df.longitude, filtered_df.latitude))
    return geodf

## Function to schedule the task

In [15]:
def scheduled_task():
    gdf=fetch_and_process_fire_data()
    # print(gdf.tail())
    # Clear previous output and display the updated map
    clear_output(wait=True)
    
    geo_data = GeoData(geo_dataframe = gdf)
    # Add new data to the leaflet map
    map.add(geo_data)
    
    # Display the updated map
    display(map)


Map(center=[37.0902, -95.7129], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

## Set up Scheduler

In [ ]:
scheduler = BackgroundScheduler()
scheduler.add_job(scheduled_task, 'interval', minutes=1)  # Fetch every minute
scheduler.start()
# Display the initial map
scheduled_task()

In [ ]:
# Keep the Jupyter Notebook running so that APScheduler can execute tasks
import time
while True:
    time.sleep(1)